In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os, textwrap
from openai import OpenAI
import getpass

# Prompt user for a password (input hidden)
OPENAI_API_KEY = getpass.getpass("Enter your password: ")


Enter your password: ··········


In [3]:
!pip install tenacity==9.0.0
!pip install langchain==0.3.12
!pip install langchain-openai==0.2.12
!pip install langchain_community==0.3.12
!pip install langgraph==0.2.59
!pip install pysqlite3-binary==0.5.4
!pip install langchain_chroma==0.1.4
!pip install pandas==2.2.3
!pip install pypdf==5.1.0
!pip install nbformat==5.10.4

  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.5.0
    Uninstalling tenacity-8.5.0:
      Successfully uninstalled tenacity-8.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.17.0 requires tenacity<9.0.0,>=8.0.0, but you have tenacity 9.0.0 which is incompatible.
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 16.8 MB/s eta 0:00:0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 114.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.5 MB/s eta 0:00:00


# Setup with ChatOpenAI and OpenAIEmbeddings

In [3]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os

# ✅ Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# 🔹 LLM: ChatOpenAI (non-Azure)
model = ChatOpenAI(
    model="gpt-4o-mini",   # or "gpt-4o"
    temperature=0
)

# 🔹 Embeddings: OpenAIEmbeddings
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large"
)


In [4]:
import pandas as pd
# Load the CSV once globally to avoid reloading each time the tool runs
PRODUCT_PRICING_PATH = "/content/data/Laptop pricing.csv"
product_pricing_df = pd.read_csv(PRODUCT_PRICING_PATH)
print(product_pricing_df)

            Name  Price  ShippingDays
0  AlphaBook Pro   1499             2
1     GammaAir X   1399             7
2  SpectraBook S   2499             7
3   OmegaPro G17   2199            14
4  NanoEdge Flex   1699             2


# Product Pricing tool

In [5]:

from langchain_core.tools import tool

@tool
def get_laptop_price(laptop_name: str) -> int:
    """
    Return the price of a laptop by name.

    Parameters
    ----------
    laptop_name : str
        The name or partial name of the laptop.

    Returns
    -------
    int
        The price if found, otherwise -1.
    """
    # Perform a case-insensitive substring match on the "Name" column
    matches = product_pricing_df[
        product_pricing_df["Name"].str.contains(laptop_name, case=False, na=False)
    ]

    # Return the first price match if available, otherwise -1
    return int(matches["Price"].iloc[0]) if not matches.empty else -1



In [6]:
#Test the tool. Before running the test, comment the @tool annotation
print(get_laptop_price("alpha"))
print(get_laptop_price("testing"))

1499
-1


/tmp/ipython-input-2456513083.py:2: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(get_laptop_price("alpha"))


# Product Features Retrieval Tool

In [7]:
!pip install pypdf

In [8]:
# Ensure Chroma works with pysqlite3 (useful in some notebook environments)
__import__("pysqlite3")
import sys
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")

from pathlib import Path

from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ---------- Config ----------
PRODUCT_PDF_PATH = Path("data") / "Laptop product descriptions.pdf"

# ---------- Load & Chunk ----------
loader = PyPDFLoader(str(PRODUCT_PDF_PATH))
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=256,
)
splits = text_splitter.split_documents(docs)

# ---------- Vector Store ----------
prod_feature_store = Chroma.from_documents(
    documents=splits,
    embedding=embedding,  # assumes `embedding` (OpenAIEmbeddings) is already defined
)

# ---------- Retriever Tool ----------
get_product_features = create_retriever_tool(
    prod_feature_store.as_retriever(search_kwargs={"k": 1}),
    name="Get_Product_Features",
    description=(
        "Retrieve detailed laptop product information, including available models and "
        "their CPU, memory, storage, design, and key advantages."
    ),
)


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [9]:
get_product_features.invoke({"query": "Lenovo ThinkPad X1 Carbon"})


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


'Fictional Laptop Descriptions\nAlphaBook Pro\nThe AlphaBook Pro is a sleek ultrabook with a 12th Gen Intel i7 processor, 16GB of DDR4 RAM,\nand a fast 1TB SSD. Ideal for professionals on the go, this laptop offers an impressive blend of\npower and portability.\nGammaAir X\nGammaAir X combines an AMD Ryzen 7 processor with 32GB of DDR4 memory and a 512GB\nNVMe SSD. Its thin and light form factor makes it perfect for users who need high performance in a\nportable design.\nSpectraBook S\nDesigned for power users, SpectraBook S features an Intel Core i9 processor, 64GB RAM, and a\nmassive 2TB SSD. This workstation-class laptop is perfect for intensive tasks like video editing and\n3D rendering.\nOmegaPro G17\nOmegaPro G17 is a gaming powerhouse with a Ryzen 9 5900HX CPU, 32GB RAM, and a 1TB\nSSD. Designed for gamers, it features a 17-inch display with a high refresh rate and powerful\ngraphics card for the best gaming experience.\nNanoEdge Flex'

In [10]:
#Test the product feature store
print(prod_feature_store.as_retriever().invoke("Tell me about the AlphaBook Pro") )

[Document(metadata={'page': 0, 'source': 'data/Laptop product descriptions.pdf'}, page_content='Fictional Laptop Descriptions\nAlphaBook Pro\nThe AlphaBook Pro is a sleek ultrabook with a 12th Gen Intel i7 processor, 16GB of DDR4 RAM,\nand a fast 1TB SSD. Ideal for professionals on the go, this laptop offers an impressive blend of\npower and portability.\nGammaAir X\nGammaAir X combines an AMD Ryzen 7 processor with 32GB of DDR4 memory and a 512GB\nNVMe SSD. Its thin and light form factor makes it perfect for users who need high performance in a\nportable design.\nSpectraBook S\nDesigned for power users, SpectraBook S features an Intel Core i9 processor, 64GB RAM, and a\nmassive 2TB SSD. This workstation-class laptop is perfect for intensive tasks like video editing and\n3D rendering.\nOmegaPro G17\nOmegaPro G17 is a gaming powerhouse with a Ryzen 9 5900HX CPU, 32GB RAM, and a 1TB\nSSD. Designed for gamers, it features a 17-inch display with a high refresh rate and powerful\ngraphics c

In [11]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create a System prompt to provide a persona to the chatbot
system_prompt = SystemMessage("""
    You are professional chatbot that answers questions about laptops sold by your company.
    To answer questions about laptops, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses.
    """
)

#Create a list of tools available
tools = [get_laptop_price, get_product_features]

#Create memory across questions in a conversation (conversation memory)
checkpointer=MemorySaver()

#Create a Product QnA Agent. This is actual a graph in langGraph
product_QnA_agent=create_react_agent(
                                model=model, #LLM to use
                                tools=tools, #List of tools to use
                                state_modifier=system_prompt, #The system prompt
                                debug=False, #Debugging turned on if needed
                                checkpointer=checkpointer #For conversation memory
)

In [12]:
#Setup chatbot
import uuid
#To maintain memory, each request should be in the context of a thread.
#Each user conversation will use a separate thread ID
config = {"configurable": {"thread_id": uuid.uuid4()}}

#Test the agent with an input
inputs = {"messages":[
                HumanMessage("What are the features and pricing for GammaAir?")
            ]}

#Use streaming to print responses as the agent  does the work.
#This is an alternate way to stream agent responses without waiting for the agent to finish
for stream in product_QnA_agent.stream(inputs, config, stream_mode="values"):
    message=stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()


================================ Human Message =================================

What are the features and pricing for GammaAir?
================================== Ai Message ==================================
Tool Calls:
  Get_Product_Features (call_wAPE0n40c0XHh8AtOk1W9pqE)
 Call ID: call_wAPE0n40c0XHh8AtOk1W9pqE
  Args:
    query: GammaAir
  get_laptop_price (call_TJCqVzadt1tlRCpw7q3eGd2T)
 Call ID: call_TJCqVzadt1tlRCpw7q3eGd2T
  Args:
    laptop_name: GammaAir
================================= Tool Message =================================
Name: get_laptop_price

1399
================================== Ai Message ==================================

The **GammaAir X** features the following specifications:

- **Processor**: AMD Ryzen 7
- **Memory**: 32GB of DDR4 RAM
- **Storage**: 512GB NVMe SSD
- **Design**: Thin and light form factor, perfect for high performance in a portable design.

The price for the GammaAir X is **$1,399**. If you have any more questions or need further ass

In [18]:
# 🧾 AFTER streaming: show the full conversation stored in the graph state
print("\n====== Full conversation history for this thread ======\n")

state = product_QnA_agent.get_state(config)  # <-- key line

# `state.values["messages"]` is the full message list
for msg in state.values["messages"]:
    msg.pretty_print()


====== Full conversation history for this thread ======

================================ Human Message =================================

hi
================================== Ai Message ==================================

Hello! How can I assist you today? If you have any questions about our laptops, feel free to ask!
================================ Human Message =================================

what is the sky color during the day
================================== Ai Message ==================================

The sky typically appears blue during the day due to the scattering of sunlight by the Earth's atmosphere. If you have any questions about laptops or need assistance with something specific, just let me know!
================================ Human Message =================================

how many laptops you have in your catelogue
================================== Ai Message ==================================

I don't have the exact number of laptops in our catalog, bu

# Multi turn conversation

In [14]:
import uuid
from langchain_core.messages import HumanMessage

# 🗣️ Simulate a conversation between the user and the Agentic Chatbot
user_inputs = [
    "Hi there!",
    "I'm planning to purchase a new laptop.",
    "Can you show me which laptop models are available?",
    "What are the specifications of the SpectraBook?",
    "How much does that one cost?",
    "Please share similar details for the OmegaPro as well.",
    "Do you have any information about a model called AcmeRight?",
    "Thanks for your help!"
]

# 🧩 Create a new thread (unique chat session)
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

print("\n💬 Starting conversation with the Agentic Chatbot...\n")

# 🪄 Simulate the back-and-forth conversation
for i, user_text in enumerate(user_inputs, start=1):
    print(f"🧍‍♂️ USER [{i}]: {user_text}\n")

    # Wrap user message
    user_message = {"messages": [HumanMessage(content=user_text)]}

    # Get agent response
    ai_response = product_QnA_agent.invoke(user_message, config=config)
    agent_reply = ai_response["messages"][-1].content.strip()

    print(f"🤖 AGENT: {agent_reply}\n")
    print("-" * 60)

print("\n✅ Conversation ended.")



💬 Starting conversation with the Agentic Chatbot...

🧍‍♂️ USER [1]: Hi there!

🤖 AGENT: Hello! How can I assist you today?

------------------------------------------------------------
🧍‍♂️ USER [2]: I'm planning to purchase a new laptop.

🤖 AGENT: That's great! Do you have any specific requirements or preferences for the laptop you're looking for?

------------------------------------------------------------
🧍‍♂️ USER [3]: Can you show me which laptop models are available?

🤖 AGENT: Could you please provide me with a specific brand or type of laptop you're interested in? This will help me find the available models for you.

------------------------------------------------------------
🧍‍♂️ USER [4]: What are the specifications of the SpectraBook?

🤖 AGENT: The SpectraBook S is designed for power users and comes with the following specifications:

- **Processor**: Intel Core i9
- **Memory**: 64GB RAM
- **Storage**: 2TB SSD
- **Ideal for**: Intensive tasks like video editing and 3D rend

# Multi-turn live conversation

In [19]:
import uuid
from langchain_core.messages import HumanMessage

# 🧠 Option 1: Predefined conversation list
user_inputs = [
    "Hi there!",
    "I'm planning to purchase a new laptop.",
    "Can you show me which laptop models are available?",
    "What are the specifications of the SpectraBook?",
    "How much does that one cost?",
    "Please share similar details for the OmegaPro as well.",
    "Do you have any information about a model called AcmeRight?",
    "Thanks for your help!"
]

# ✅ Restore built-in input in case it was overwritten earlier
import builtins
input = builtins.input

# 🧩 Create a new chat session
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

print("\n💬 Starting conversation with the Agentic Chatbot...\n")
print("Type 'exit' anytime to end the chat.\n")

# Ask user which mode to use
mode = input("Enter '1' for predefined conversation, or '2' for interactive chat: ").strip()

if mode == "1":
    # -----------------------------
    # Predefined conversation mode
    # -----------------------------
    for i, user_text in enumerate(user_inputs, start=1):
        print(f"\n🧍‍♂️ USER [{i}]: {user_text}")
        user_message = {"messages": [HumanMessage(content=user_text)]}
        ai_response = product_QnA_agent.invoke(user_message, config=config)
        agent_reply = ai_response["messages"][-1].content.strip()
        print(f"🤖 AGENT: {agent_reply}")
        print("-" * 60)

else:
    # -----------------------------
    # Interactive mode
    # -----------------------------
    turn = 1
    while True:
        user_text = input(f"\n🧍‍♂️ USER [{turn}]: ").strip()
        if user_text.lower() in {"exit", "quit", "bye"}:
            print("\n👋 Conversation ended.")
            break
        user_message = {"messages": [HumanMessage(content=user_text)]}
        ai_response = product_QnA_agent.invoke(user_message, config=config)
        agent_reply = ai_response["messages"][-1].content.strip()
        print(f"🤖 AGENT: {agent_reply}")
        print("-" * 60)
        turn += 1



💬 Starting conversation with the Agentic Chatbot...

Type 'exit' anytime to end the chat.

Enter '1' for predefined conversation, or '2' for interactive chat: 2

🧍‍♂️ USER [1]: hey
🤖 AGENT: Hello! How can I assist you today? If you have any questions about our laptops, feel free to ask!
------------------------------------------------------------

🧍‍♂️ USER [2]: suggest me the best laptop i should look for
🤖 AGENT: Could you please provide me with some details about your needs? For example, are you looking for a laptop for gaming, business, graphic design, or general use? Additionally, any preferences regarding budget or specific features would be helpful!
------------------------------------------------------------

🧍‍♂️ USER [3]: business
🤖 AGENT: Here are some excellent business laptops you might consider:

1. **AlphaBook Pro**
   - **Processor:** 12th Gen Intel i7
   - **RAM:** 16GB DDR4
   - **Storage:** 1TB SSD
   - **Description:** A sleek ultrabook ideal for professionals on t

# Different Session Id's

In [20]:
from langchain_core.messages import HumanMessage
import uuid

# ------------------------------------------
# 💬 Utility: Execute prompt & show conversation context
# ------------------------------------------
def execute_prompt(user_label, config, prompt):
    inputs = {"messages": [HumanMessage(prompt)]}
    ai_response = product_QnA_agent.invoke(inputs, config=config)

    # Extract the full conversation state for this user
    state = product_QnA_agent.get_state(config)
    conversation = state.values["messages"]

    # 🧠 Insightful, context-aware printing
    print(f"\n{'='*60}")
    print(f"🧩 Conversation Thread for: {user_label}")
    print(f"Prompt: {prompt}")
    print(f"{'-'*60}")
    for msg in conversation:
        role = getattr(msg, "type", "unknown").capitalize()
        print(f"{role}: {msg.content}")
    print(f"{'='*60}\n")

    # Highlight only the latest AI response at the end
    latest_ai = ai_response["messages"][-1].content
    print(f"🤖 {user_label} → AI says:\n{latest_ai}\n")

# ------------------------------------------
# 🧑‍💻 Setup independent conversation threads for two users
# ------------------------------------------
config_user1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_user2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

# ------------------------------------------
# 🧩 Test both sessions to show independent memory retention
# ------------------------------------------
execute_prompt("USER 1", config_user1, "Tell me about the features of SpectraBook")
execute_prompt("USER 2", config_user2, "Tell me about the features of GammaAir")

execute_prompt("USER 1", config_user1, "What is its price?")
execute_prompt("USER 2", config_user2, "What is its price?")



🧩 Conversation Thread for: USER 1
Prompt: Tell me about the features of SpectraBook
------------------------------------------------------------
Human: Tell me about the features of SpectraBook
Ai: 
Tool: Fictional Laptop Descriptions
AlphaBook Pro
The AlphaBook Pro is a sleek ultrabook with a 12th Gen Intel i7 processor, 16GB of DDR4 RAM,
and a fast 1TB SSD. Ideal for professionals on the go, this laptop offers an impressive blend of
power and portability.
GammaAir X
GammaAir X combines an AMD Ryzen 7 processor with 32GB of DDR4 memory and a 512GB
NVMe SSD. Its thin and light form factor makes it perfect for users who need high performance in a
portable design.
SpectraBook S
Designed for power users, SpectraBook S features an Intel Core i9 processor, 64GB RAM, and a
massive 2TB SSD. This workstation-class laptop is perfect for intensive tasks like video editing and
3D rendering.
OmegaPro G17
OmegaPro G17 is a gaming powerhouse with a Ryzen 9 5900HX CPU, 32GB RAM, and a 1TB
SSD. Desig

# ----------------------------------------------------
# 🧾 Print the overall conversation history for each user
# ----------------------------------------------------

In [21]:

def print_full_conversation(user_label, config):
    state = product_QnA_agent.get_state(config)
    conversation = state.values["messages"]

    print(f"\n{'='*70}")
    print(f"📜 Overall Conversation for {user_label}")
    print(f"{'='*70}")
    for msg in conversation:
        role = getattr(msg, "type", "unknown").capitalize()
        print(f"{role}: {msg.content}")
    print(f"{'='*70}\n")

# ✅ Show all messages exchanged in each thread
print_full_conversation("USER 1", config_user1)
print_full_conversation("USER 2", config_user2)


📜 Overall Conversation for USER 1
Human: Tell me about the features of SpectraBook
Ai: 
Tool: Fictional Laptop Descriptions
AlphaBook Pro
The AlphaBook Pro is a sleek ultrabook with a 12th Gen Intel i7 processor, 16GB of DDR4 RAM,
and a fast 1TB SSD. Ideal for professionals on the go, this laptop offers an impressive blend of
power and portability.
GammaAir X
GammaAir X combines an AMD Ryzen 7 processor with 32GB of DDR4 memory and a 512GB
NVMe SSD. Its thin and light form factor makes it perfect for users who need high performance in a
portable design.
SpectraBook S
Designed for power users, SpectraBook S features an Intel Core i9 processor, 64GB RAM, and a
massive 2TB SSD. This workstation-class laptop is perfect for intensive tasks like video editing and
3D rendering.
OmegaPro G17
OmegaPro G17 is a gaming powerhouse with a Ryzen 9 5900HX CPU, 32GB RAM, and a 1TB
SSD. Designed for gamers, it features a 17-inch display with a high refresh rate and powerful
graphics card for the best 